In [3]:
!pip install undetected_chromedriver

     |████████████████████████████████| 58 kB 686 kB/s 
     |████████████████████████████████| 97 kB 1.3 MB/s 
  Created wheel for undetected-chromedriver: filename=undetected_chromedriver-3.1.7-py3-none-any.whl size=45412 sha256=a73ec35ad87d4446d02922d88a5f7562a664e59182d0aeac576be29fd335939d
  Stored in directory: /Users/tranxuanhuy/Library/Caches/pip/wheels/5f/35/f2/a6f20630c5c90a565e082ab1eea19a7a6696fa09f66c7aab01
Successfully built undetected-chromedriver


In [6]:
import random
import time
from datetime import datetime
from IPython.display import clear_output
import pandas as pd
from bs4 import BeautifulSoup 
from selenium import webdriver
from  lxml import etree
import json 
import os
from tqdm import tqdm
import traceback
import undetected_chromedriver as uc
import traceback
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService

In [38]:
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
url = "https://alonhadat.com.vn/ban-nha-dep-quan-hai-ba-trung-50m-5-tang-5-5-ty-12173405.html"
driver.get(url)
bs = BeautifulSoup(driver.page_source)

page = etree.HTML(str(bs))
price = page.xpath("""//*[@id="left"]/div[1]/div[3]/span[1]/span[2]""")[0].text
print(price)


 5,5 tỷ 


In [135]:
TIME=0.5

class crawling:
    
    def __init__(self,):
        self.home_page = "https://alonhadat.com.vn/nha-dat/can-ban/nha-dat/1/ha-noi.html"
        self.root = "https://alonhadat.com.vn"
        self.page = 1

    
    def get_pages(self, page_source):
        bs = BeautifulSoup(page_source)
        page = etree.HTML(str(bs))

        elements = bs.find_all("div", {"class": "ct_title"})
        data = []
        try:
            for feature in elements:
                feature = feature.find_all('a', href=True)
                data.append(feature[0]['href'])
        except Exception as e:
            print(e)
        return [self.root+link for link in data]
    
    def next_page(self):
        self.page += 1
        return self.home_page[:-5] + f"/trang--{self.page}.html"
        # return  self.home_page + "f/p{self.page}"

    def get_id(self, page_source):
        bs = BeautifulSoup(page_source)
        page = etree.HTML(str(bs))
        address = page.xpath("""//*[@id="left"]/div[1]/div[5]/div[2]/table/tbody/tr[1]""")[0]
        print(address)
        return address.getchildren()[1].text
    
    def gather(self, page_source):
        data = {}
        bs = BeautifulSoup(page_source)
        page = etree.HTML(str(bs))
        
        # get price
        # price = bs.find_all('span', {'class': 'price'})
        price = page.xpath("""//*[@id="left"]/div[1]/div[3]/span[1]/span[2]""")[0].text
        price = price.replace(",", ".")
        data['price'] = price

        # get area
        area = page.xpath("""//*[@id="left"]/div[1]/div[3]/span[2]/span[2]""")[0].text
        data['area'] = area
        
        # get address
        address = page.xpath("""//*[@id="left"]/div[1]/div[4]/span[2]""")[0].text
        address = address.replace(",", ".")
        data['address'] = address

        # get table information
        table = bs.find_all('div', {'class': 'infor'})[0]
        tr_instances = table.find_all('tr')
        for i in range(len(tr_instances)):
            td_instances = tr_instances[i].find_all('td')
            for j in range(3):
                data[str(td_instances[2*j].text).strip()] = str(td_instances[2*j+1].text)
        return data

    def run(self, start, num_of_pages):
        service = ChromeService(executable_path=ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service)
        url = self.home_page
        self.page = 1
        if start != 1:
            self.page = start
            url = self.home_page[:-5] + f"/trang--{self.page}.html"
        if os.path.exists("data/index.json"):
            with open("data/index.json") as f:
                index = set(json.load(f))
        else:
            index = set()
            
        with open("log.txt", "w") as f:
            pass
        
        dataset = []
        for i in range(num_of_pages):
            num = 10
            pages = []
            while num > 0:
                try:
                    driver.get(url)
                    time.sleep(TIME)
                    pages = self.get_pages(driver.page_source)
                    driver.delete_all_cookies()
                    break
                except:
                    num -= 1
            if num <= 0:
                url = self.next_page()
                continue

            for page in tqdm(pages):
                num = 1
                while num > 0:
                    try:
                        driver.get(page)
                        time.sleep(TIME)
                        id = self.get_id(driver.page_source)
                        if id in index: 
                            print(id, "exist")
                            break

                        data = self.gather(driver.page_source)
                        
                        if data["Mã tin"] not in index:
                            dataset.append(data)
                            index.add(data["Mã tin"])
                        with open("log.txt", "a") as f:
                            f.write("Success:" + page +"\n")
                        driver.delete_all_cookies()
                        break
                    except Exception as e:
                        print(e, "unknown")
                        with open("log.txt", "a") as f:
                            f.write(f"Excution{5-num}: {page}"+"\n") 
                            f.write(traceback.format_exc()+"\n")
                        
                        with open("failed.txt", "a") as f:
                            f.write(f"Excution{5-num}: {page}"+"\n") 
                            f.write(traceback.format_exc()+"\n")
                        driver.delete_all_cookies()
                        num -= 1
            url = self.next_page()
        if len(dataset) != 0:
            df = pd.DataFrame(dataset)
            df.to_csv(f"data/{start}_{start+num_of_pages}_{time.time()}.csv", index=False)
        with open("data/index.json", "w") as f:
            json.dump(list(index), f)
        driver.close()
        
craw = crawling()

In [137]:

# Cần bán: 11353 (the last page)
# Cho thuê: 1211 (the last page)

for i in range(1, 3):
    craw.run(i, 1)
    clear_output()